## 1. Enable Auto Reloading

In [1]:
%load_ext autoreload
%autoreload 2

## 2. Load Packages & Import External Functions

In [2]:
import json
import gzip
import numpy as np
from copy import deepcopy
import regex as re

from excel_processor import ExcelSet

## 3. Load All Files (Excel, Input & Output JSON)

In [3]:
def read_json_gz(fpath):
    with gzip.open(fpath, 'r') as fin:
        json_bytes = fin.read()
    json_str = json_bytes.decode('utf-8')
    data = json.loads(json_str)
    return data
    
# generator is the only excel file we need so far
excel_set = ExcelSet(fnames=['Generator']) 

# load json
# make sure you first navigate to the uc_outputs folder
# and unzip the .gz file to GridViewSampleResult.json
uc_output_dir = './uc_outputs'
uc_output_fname = 'GridViewSampleResult'
uc_output_ext = '.json.gz'
uc_output_fpath = f'{uc_output_dir}/{uc_output_fname}{uc_output_ext}'
data = read_json_gz(uc_output_fpath)

# we need some information from the UC.jl input
# make sure you first navigate to the uc_jsons folder
# and unzip the .gz file to GridViewSampleResult.json
uc_input_dir = './uc_jsons'
uc_input_fname = 'GridViewSample'
uc_input_ext = '.json.gz'
uc_input_fpath = f'{uc_input_dir}/{uc_input_fname}{uc_input_ext}'
data_input = read_json_gz(uc_input_fpath)

In [4]:
# UC.jl supports multiple scenarios
# the most common case is a single scenario
# I do not think we will ever encounter multiple scenarios
single_scenario = "Thermal production (MW)" in data

## 4. Report SubType Total Energy & Production Cost by Years

In [5]:
# if the dictionary has the key, then add the value to it
# otherwise, create that key with the current value
def create_or_add(dictionary, key, val):
    if key in dictionary:
        dictionary[key] += val
    else:
        dictionary[key] = val
        
# make sure the dictionary has the key of the year
def ensure_year_key(dictionary, year, default):
    if not year in dictionary:
        dictionary[year] = default

In [6]:
# get the gen_key and year
def parse_gen_name(gen_name):
    r = re.match("^g(\d+)_(\d+)$", gen_name)
    gen_key = int(r.group(1))
    year = int(r.group(2))
    return gen_key, year

In [7]:
thermal_prods = data["Thermal production (MW)"]
thermal_costs = data["Thermal production cost ($)"]

# prepare the result dictionary
# res_prod = {
#   2010: {"CC": 400, "MT": 300}, 
#   2012: {"CC": 40, "MT": 30}, 
# }
res_prod = {}
res_cost = {}

for gen_name, prod in thermal_prods.items():
    # step 1: get necessary information of the generator
    # generator name follows the pattern "g<GEN_KEY>_<YEAR>"
    gen_key, year = parse_gen_name(gen_name)
    # query the excel_set to get the sub type
    sub_type = excel_set.query_and_retrieve("Generator", "GeneratorKey", [gen_key], "SubType")
    
    # step 2: get the numerical values we need
    # sum to get the total prod & costs
    total_prod = sum(prod)
    total_cost = sum(thermal_costs[gen_name])
    
    # step 3: update value in the result
    # make sure the year is in the 
    ensure_year_key(res_prod, year, {})
    ensure_year_key(res_cost, year, {})
    # create or add the sub_type to the result
    create_or_add(res_prod[year], sub_type, total_prod)
    create_or_add(res_cost[year], sub_type, total_cost)

print(res_prod)
print(res_cost)

{2010: {'CC': 67808729.78329182, 'ST': 158591039.66374397, 'GT': 6955031.979120016, 'NP': 21243000.0, 'CS': 11436179.983291626, 'AB': 2674428.0016708374, 'EQ': 5255400.0}}
{2010: {'CC': 3274136034.9175377, 'ST': 6738128779.549359, 'GT': 529390063.94892675, 'NP': 133552089.58906174, 'CS': 582096511.7600487, 'AB': 33865196.98625255, 'EQ': 331128671.33502877}}


## 5. Report Hydro & Profiled Resource Curtailment

In [8]:
profiled_prods = data["Profiled production (MW)"]

# example of how res_profiled_curtail looks
'''
{
    2010: {
        "Unused hydro generators": [ID1, ID2, ...],
        "Unused profiled generators": [ID1, ID2, ...],
        "Total annual profiled curtailment (MW)": 10000,
        "Total annual hydro curtailment (MW)": 500,
        "Total hourly profiled curtailment (MW)": [70, 20, ...],
        "Total hourly hydro curtailment (MW)": [70, 20, ...],
    }
}
'''
res_profiled_curtail = {}

for gen_name, prod in profiled_prods.items():
    # step 1: get necessary information of the generator
    # generator name follows the pattern "g<GEN_KEY>_<YEAR>"
    gen_key, year = parse_gen_name(gen_name)
    # query the excel_set to get the profiled type
    gen_type = excel_set.query_and_retrieve("Generator", "GeneratorKey", [gen_key], "GeneratorType")
    
    # step 2: update value in the result
    ensure_year_key(res_profiled_curtail, year, default={
        "Unused hydro generators": [],
        "Unused profiled generators": [],
        "Total annual profiled curtailment (MW)": 0,
        "Total annual hydro curtailment (MW)": 0,
        "Total hourly profiled curtailment (MW)": [0.0] * len(prod),
        "Total hourly hydro curtailment (MW)": [0.0] * len(prod),
    })
    
    # step 3: extract all the lists to avoid long line code
    # the update follows "pass-by-reference" for lists 
    curr_year_curt = res_profiled_curtail[year]
    unused_profiled = curr_year_curt["Unused profiled generators"]
    unused_hydro = curr_year_curt["Unused hydro generators"]
    hourly_profiled_curt = curr_year_curt["Total hourly profiled curtailment (MW)"]
    hourly_hydro_curt = curr_year_curt["Total hourly hydro curtailment (MW)"]
    
    # step 3: process unused energy
    if sum(prod) < 1e-3:
        unused_profiled.append(gen_key)
        if int(gen_type) == 4:
            unused_hydro.append(gen_key)
    
    # step 4: get hourly and annual curtailment
    max_power = data_input["Generators"][gen_name]["Maximum power (MW)"]
    for t in range(len(prod)):
        # max_power may be a number or a list
        if isinstance(max_power, (int, float)):
            max_power_t = max_power
        else:
            max_power_t = max_power[t]
        # calc the power curtailed, and correct any numeric precision issue
        power_curt = max_power_t - prod[t]
        if abs(power_curt) < 1e-5:
            power_curt = 0.0
        # add it to the list and total number
        hourly_profiled_curt[t] += power_curt
        curr_year_curt["Total annual profiled curtailment (MW)"] += power_curt
        if int(gen_type) == 4:
            hourly_hydro_curt[t] += power_curt
            curr_year_curt["Total annual hydro curtailment (MW)"] += power_curt    

# print the result verify if the hourly and total reports match
for year, info in res_profiled_curtail.items():
    print(f"Year {year}:")
    print(f"-- Total annual profiled curt (MW): {info['Total annual profiled curtailment (MW)']}")
    print(f"-- Total annual hydro curt (MW): {info['Total annual hydro curtailment (MW)']}")
    print(f"-- Sum of hourly profiled curt (MW): {sum(info['Total hourly profiled curtailment (MW)'])}")
    print(f"-- Sum of hourly hydro curt (MW): {sum(info['Total hourly hydro curtailment (MW)'])}")

Year 2010:
-- Total annual profiled curt (MW): 0.0
-- Total annual hydro curt (MW): 0.0
-- Sum of hourly profiled curt (MW): 0.0
-- Sum of hourly hydro curt (MW): 0.0


## 6. Report the Load Curtailment

In [9]:
def parse_bus_name(bus_name):
    # eg: MEGA_2010
    r = re.match("^(\w+)_(\d+)$", bus_name)
    area_name = r.group(1)
    year = int(r.group(2))
    return area_name, year

In [10]:
load_curtail = data["Load curtail (MW)"]

res_curtail = {}

for bus_name, curtails in load_curtail.items():
    area_name, year = parse_bus_name(bus_name)
    total_curtail = sum(curtails)
    
    ensure_year_key(res_curtail, year, {})
    create_or_add(res_curtail[year], area_name, total_curtail)

print(res_curtail)

{2010: {'MEGA': 2334293315.342141}}


## 7. Report the LMP

In [11]:
# to do, this is a large LP to solve
res_hourly_lmp = {}

## 8. Prepare the final result

In [12]:
final_res = {
    "Total production by sub-type (MW)": res_prod,
    "Total production cost by sub-type ($)": res_cost,
    "Hourly LMP curve ($)": res_hourly_lmp,
    "Curtailed profiled generators": res_profiled_curtail,
    "Total load curtail (MW)": res_curtail,
}

uc_processed_output_name = f'Processed_{uc_output_fname}.json'
uc_processed_output_fpath = f'{uc_output_dir}/{uc_processed_output_name}'

with open(uc_processed_output_fpath, 'w', encoding='utf8') as write_file:
    json.dump(final_res, write_file, indent=4)
    print(f"@info: the file has been saved to {uc_processed_output_fpath}")

@info: the file has been saved to ./uc_outputs/Processed_GridViewSampleResult.json
